In [3]:
from mxnet import nd, autograd
import random

true_W = nd.array([2, -3.4]).reshape(2, 1)
true_b = -4.2
num_inputs = 2
num_samples = 1000
features = nd.random.normal(scale=1, shape=(num_samples, num_inputs))
labels = nd.dot(features, true_W) + true_b
labels += nd.random.normal(scale = 0.01, shape=labels.shape)

In [9]:
from mxnet.gluon import data as gdata

batch_size = 10
dataset = gdata.ArrayDataset(features, labels)
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)
for X, y in data_iter:
    print(X, y)
    break


[[-1.5666938   0.61245096]
 [ 1.0494479   0.14226237]
 [ 1.8858474   0.07053453]
 [-0.7651942   0.16437985]
 [-0.03142362 -2.3463464 ]
 [-0.5893647  -0.89429986]
 [ 0.11958748  1.0943946 ]
 [ 0.7848637   0.09315785]
 [-0.26842585  0.09316857]
 [-1.3158178   0.7985707 ]]
<NDArray 10x2 @cpu(0)> 
[[-9.408498  ]
 [-2.5718586 ]
 [-0.66279685]
 [-6.28455   ]
 [ 3.7215796 ]
 [-2.3513904 ]
 [-7.678594  ]
 [-2.9502563 ]
 [-5.068206  ]
 [-9.545964  ]]
<NDArray 10x1 @cpu(0)>


In [5]:
from mxnet.gluon import nn
network = nn.Sequential()
network.add(nn.Dense(1))

In [13]:
from mxnet import init
network.initialize(init.Normal(sigma = 0.01))
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss()

/home/breeze/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:813: UserWarning: Parameter 'dense0_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/breeze/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:813: UserWarning: Parameter 'dense0_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)


In [15]:
from mxnet import gluon
trainer = gluon.Trainer(network.collect_params(), 'sgd', {'learning_rate':0.03})
num_epochs = 5
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(network(X), y).mean()
        l.backward()
        trainer.step(1)
    l = loss(network(features), labels)
    print('epoch %d, loss %f' % (epoch, l.mean().asscalar()))

epoch 1, loss 0.032673
epoch 2, loss 0.000114
epoch 3, loss 0.000049
epoch 4, loss 0.000049
epoch 5, loss 0.000049


In [17]:
dense = network[0]
true_W, dense.weight.data()

(
 [[ 2. ]
  [-3.4]]
 <NDArray 2x1 @cpu(0)>, 
 [[ 1.9999746 -3.4000585]]
 <NDArray 1x2 @cpu(0)>)

In [18]:
true_b, dense.bias.data()

(-4.2, 
 [-4.200469]
 <NDArray 1 @cpu(0)>)